In [ ]:
%config Completer.use_jedi=False

import sys
import os
import glob
import matplotlib.pyplot as plt
import numpy as np
sys.path.append('../../')

def get_line_type(fname):
    if (fname in ['microbes','monomers','substrates','degradation_enzymes']):
        res = '-'
    else:
        res = '--'
    return res
from refsimu_singletrait import reload_simulation
from refsimu_singletrait import microbe_pop_on

In [ ]:
microbe_pop_on()

In [ ]:
# base_dir = './BIDULETESTMEM/' # CHANGE FOR YOUR OWN 
base_dir = './MACHIN/'
sdirs = [f for f in glob.glob(os.path.join(base_dir,'*')) if os.path.isdir(f)]
dlists = {}
for sdir in sdirs:
#     print(sdir)
    skey = '_'.join(os.path.basename(sdir).split('_')[1:3])
#     print(skey)
    dlist = glob.glob(os.path.join(sdir,'replica_*'))
    dlist = [d for d in dlist if os.path.isdir(d)]
    dlists[skey] = dlist

# dictionnary dlists is keyed by names of the base_dir subdirectories ( CELLULOSE conditions), the values are lists
# of the subdirectories containing the various replica
# for k,d in dlists.items():
#     print('*'*20,'\n',k)
#     for i,r in enumerate(d):
#         print(i,r)

In [ ]:
rep_dir = dlists['CELLULOSEX2_NoMut'][0] # select condition and replica 
print('Reloading from {}'.format(rep_dir))
eco = reload_simulation(rep_dir,-1)
# print(eco.grid_shape)

diagcollector = eco.get_diag_collector()
tl_files = sorted(glob.glob('{}/timelines_dump_*.hdf5'.format(rep_dir)))
# print(tl_files)
dump_file = tl_files[0]
diagcollector.load_from_dumpfile(dump_file)
tla_comp_space = diagcollector.get_timeline_arrays('space_comp_sum', tag_filter_func= lambda t: 'pool' in t and 'input' not in t)
tla_comp_space_external =  diagcollector.get_timeline_arrays('space_comp_sum', tag_filter_func= lambda t: 'pool' in t and 'input'  in t)
tla_recycling = diagcollector.get_timeline_arrays('space_comp_sum',tag_filter_func=lambda t:'stoechio_balance_recycler' in t)
tla_mortality = diagcollector.get_timeline_arrays('space_comp_sum',tag_filter_func=lambda t:'mortality_op' in t)
tla_bytype = diagcollector.get_timeline_arrays('sum_by_type')
tla_ncells = diagcollector.get_timeline_arrays('ncells')
tla_sub = diagcollector.get_timeline_arrays('space_sum',tag_filter_func=lambda t:'substrates' in t)
tla_enz = diagcollector.get_timeline_arrays('space_sum',tag_filter_func=lambda t:'degradation_enzymes' in t)
tla_mon = diagcollector.get_timeline_arrays('space_sum',tag_filter_func=lambda t:'monomers' in t)
tla_mic = diagcollector.get_timeline_arrays('space_sum',tag_filter_func=lambda t:'microbes' in t)
tla_mic_ntax = diagcollector.get_timeline_arrays('ntaxpercell')
tla_clim = diagcollector.get_timeline_arrays('Climate')
tla_mic_nind = diagcollector.get_timeline_arrays('Quanta')
tla_mic_indmass = diagcollector.get_timeline_arrays('LocalTaxIndividualMass')

In [ ]:
tmin = 365*20
tmax = tmin+(6) * 365

In [ ]:
# tla_mem = diagcollector.get_timeline_arrays('Memory')
# print(tla_mem.keys())
# for k,d in tla_mem.items():
#     fig,ax = plt.subplots(1,1,figsize=(12,3))
#     ax.plot(d['times'],d['values'] / (2**20))
#     ax.grid()
#     ax.set_ylabel('Mo')
#     ax.set_title(k)
# fig,ax = plt.subplots(1,1,figsize=(12,3))
# ax.plot(tla_mem['ru_maxrss']['times'],tla_mem['ru_maxrss']['values'])
# ax.grid()

In [ ]:
# print(eco.microbes.dispersal_op.biomass_ratios)
# print(eco.microbes.dispersal_op.saturation_thresholds)
# print(eco.microbes.dispersal_op.saturation_thresholds_dict)


In [ ]:
# for k,d in eco.microbes.metabolites.items():
#     print('*'*100)
#     print(k)
# #     print(d.active_pathway)
# #     print(d.environ_functions)
# #     print(d.external_targets)
#     print(d.ref_cost)

In [ ]:
# eco.microbes.get_metabolic_processes_operands('Consti',eco.get_environment(2))

In [ ]:
# truc = eco.microbes.get_metabolic_processes_targets('Induci')
# for i,o in enumerate(truc):
#     print(i,o.shape)
    

In [ ]:
# for process in ['Consti','Induci']:
#     print('*'*100)
#     print(process)
#     for k,met in eco.microbes.metabolites.items():
#         print(k)
#         print('+'*20)
#         for targ in met.get_target_types():
#             if (met.active_pathway[process][targ]):
#                 print('-'*20)
#                 print(targ)
#                 print('-'*20)
#                 print('external', met.external_targets[process][targ])
#                 print('targ',met.targets[process][targ])

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(12,4))
for itax in range(tla_mic_nind['microbes']['values'].shape[1]):
    ax.plot(tla_mic_nind['microbes']['times'],tla_mic_nind['microbes']['values'][:,itax],label='Taxon {}'.format(itax)
           )
ax.legend()
ax.grid()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(12,4))
ax.plot(tla_mic_nind['microbes']['times'],np.sum(tla_mic_nind['microbes']['values'],axis=(-2,-1)))
ax.set_xlabel('time')
ax.set_title('Nb individuals')
ax.grid();

In [ ]:
fig, axes = plt.subplots(2,1, figsize=(12,4))
for k, ax in zip(tla_clim.keys(),axes):
    ax.plot(tla_clim[k]['times'],tla_clim[k]['values'])
    ax.set_xlabel('time')
    ax.set_ylabel(k)
    ax.grid()

In [ ]:
fig, axes = plt.subplots(3,1, figsize=(16,18))
scal = 1.0 / eco.microbes.grid_size
for ia,ax in enumerate(axes):
    tmp_sum = None
    for k, d in tla_comp_space.items():
        if (tmp_sum is None):
            tmp_sum = np.zeros_like(d['values'][:,ia])
            td = d['times']
        ls = get_line_type(k)
        ax.plot(d['times'],scal*d['values'][:,ia],ls, label=k)
        tmp_sum += d['values'][:, ia]
    for k, d in tla_comp_space_external.items():
        if (tmp_sum is None):
            tmp_sum = np.zeros_like(d['values'][:,ia])
            td = d['times']
        ls = get_line_type(k)
        ax.plot(d['times'],scal*d['values'][:,ia],ls, label=k)
        tmp_sum -= d['values'][:, ia]
    ax.plot(td,scal*tmp_sum,'.',color='black',lw=0.2,alpha=0.1,label='sum internal - sum sources')
    ax.grid()
    ax.legend(fontsize=8, ncol=3,loc=1, framealpha=0.5)
    ax.set_title('Biomass {}'.format(['C','N','P'][ia]))

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(12,4))
scal = 1.0 / eco.microbes.grid_size
d = tla_comp_space['Respiration_Growth_Induci_Maint']
ax.plot(d['times'],d['values'][:,0]*scal)

ax.grid()
ax.set_title('Respiration Growth Induci Maint')

In [ ]:
fig, ax = plt.subplots(1,1)
t, dat = tla_mic_ntax['microbes']['times'],tla_mic_ntax['microbes']['values']
ax.plot(t,dat[:,0], label='mean')
ax.fill_between(t,dat[:,0]-dat[:,1],dat[:,0]+dat[:,1], alpha=0.2)
ax.plot(t,dat[:,2], label='min')
ax.plot(t,dat[:,3], label='max')
ax.grid()
ax.set_xlabel('Time')
ax.set_ylabel('Live Taxa per grid cell')
ax.legend();
plt.show()
plt.close(fig)
#

In [ ]:
fig, axes = plt.subplots(3,1, figsize=(16,18))
scal = 1.0 / eco.microbes.grid_size
for ia,ax in enumerate(axes):
    for k,d in tla_sub.items():
        for isub, subname in enumerate(eco.substrates.names):
            if (isub < 2):
                ls = '--'
            else:
                ls = '-'
            ax.plot(d['times'][tmin:tmax],scal*d['values'][:,isub,ia][tmin:tmax],ls,label=subname)
    ax.legend()
    ax.grid()
    ax.set_title('Substrate Biomass {}'.format(['C','N','P'][ia]))

In [ ]:
# tmin = 30280
# tmax = tmin + 730
# fig, ax = plt.subplots(1,1, figsize=(16,5))
# for k,d in tla_sub.items():
#     for isub, subname in enumerate(eco.substrates.names):
#         if (isub < 2):
#             ls = '--'
#         else:
#             ls = '-'
#         ssub = np.sum(d['values'][:,isub,:],axis=-1)
#         drate = np.gradient(np.log(ssub+1.0e-16))
#         drate_neg = np.zeros_like(drate)
#         drate_neg[drate < 0]= -drate[drate < 0]
#         ax.plot(d['times'][tmin:tmax],drate_neg[tmin:tmax],ls,label=subname)
# ax.legend()
# ax.grid()
# ax.set_title('Substrate Negative decay rate')

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(12,5))
scal = 1.0 / eco.microbes.grid_size
for k,d in tla_sub.items():
    ax.plot(d['times'][tmin:tmax],scal*np.sum(d['values'][tmin:tmax],axis=(-2,-1)))
ax.grid()
ax.set_title('Substrate total mass')

In [ ]:
# fig, axes = plt.subplots(3,1, figsize=(16,18))
# scal = 1.0 / eco.microbes.grid_size
# tmin = 0
# tmax = -1
# for ia,ax in enumerate(axes):
#     for k,d in tla_sub.items():

#         for isub, subname in enumerate(eco.substrates.names):
#             if (subname not in ['DeadMic','DeadEnz']):
#                 continue
#             if (isub < 2):
#                 ls = '--'
#             else:
#                 ls = '-'
#             ax.plot(d['times'][tmin:tmax],scal*d['values'][:,isub,ia][tmin:tmax],ls,label=subname)
#     ax.legend()
#     ax.grid()
#     ax.set_title('Substrate Biomass {}'.format(['C','N','P'][ia]))

In [ ]:
fig, axes = plt.subplots(2,1, figsize=(16,12))
scal = 1.0 / eco.microbes.grid_size
for ia,ax in enumerate(axes):
    for k,d in tla_enz.items():
        for ienz, enzname in enumerate(eco.degradation_enzymes.names):
            if (ienz < 2):
                ls = '--'
            else:
                ls = '-'
            ax.plot(d['times'][tmin:tmax],scal*d['values'][:,ienz,ia][tmin:tmax],ls,label=enzname)
    ax.legend()
    ax.grid()
    ax.set_title('Enzyme Biomass {}'.format(['C','N','P'][ia]))
    ax.set_ylim(0,0.25)

In [ ]:
# %matplotlib inline
# scal = 1.0 / eco.microbes.grid_size
# fig = plt.figure(figsize=(16,8))
# dat = np.sum(tla_mic['microbes']['values'][:,:,:],axis=-1)
# sdat = np.sum(dat,axis=1)
# dat = dat / sdat[:,np.newaxis]
# plt.imshow(dat.T, aspect='auto',origin='lower',interpolation='None', cmap='magma')
# plt.xlabel('time')
# plt.ylabel('taxon id')
# plt.title('Taxon Total mass Relative abundancy')
# plt.colorbar();

In [ ]:
# %matplotlib inline
# scal = 1.0 / eco.microbes.grid_size
# fig = plt.figure(figsize=(16,8))
# #plt.imshow(tla_ncells['microbes'][1].T,aspect='auto', origin='lower',interpolation='None')#
# plt.pcolormesh(tla_ncells['microbes']['values'].T *scal, shading='flat', cmap='magma')
# plt.xlabel('time')
# plt.ylabel('taxon id')
# plt.title('Taxon grid occupation fraction ')
# plt.colorbar();

In [ ]:
scal = 1.0 / eco.microbes.grid_size
for k, d in tla_bytype.items():
    mtypes = eco.microbes.get_taxa_types()
    print(mtypes)
    fig, axes = plt.subplots(3,1, figsize=(14,12))
    for ia, ax in enumerate(axes):
        ax.set_title('{} {}'.format(k, ['C','N','P'][ia]))
        for itype, typ in enumerate(mtypes):
            ax.plot(d['times'][tmin:tmax],scal*d['values'][:,itype,ia][tmin:tmax],label=typ)
        ax.grid()
        ax.legend(fontsize=9)

In [ ]:
scal = 1.0 / eco.microbes.grid_size
smin = eco.microbes.get_taxa_stoechiometry_quotas('min')[0,:]
smin_comp = np.ones_like(smin)
for ia  in range(smin_comp.shape[0]):
    smin_comp[ia] = 1.0+smin[ia]-np.sum(smin)
smax = eco.microbes.get_taxa_stoechiometry_quotas('max')[0,:]
for k, d in tla_bytype.items():
    mtypes = eco.microbes.get_taxa_types()
    fig, axes = plt.subplots(3,1, figsize=(14,12))
    dsub = tla_sub['substrates']
    ssub = np.sum(dsub['values'][:,:,:],axis=1)
    sssub = np.sum(ssub,axis=-1)
    stsub = np.divide(ssub , sssub[:,np.newaxis],where=sssub[:,np.newaxis] > 0)
    ssub_corr = ssub * np.array([0.5,1.0,1])[np.newaxis,:]
    sssub_corr = np.sum(ssub_corr,axis=-1)
    stsub_corr = np.divide(ssub_corr , sssub_corr[:,np.newaxis],where=sssub_corr[:,np.newaxis] > 0)
    for ia, ax in enumerate(axes):
        ax.set_title('{} {}'.format(k, ['C','N','P'][ia]))
        for itype, typ in enumerate(mtypes):
            if (typ != 'bacteria'):
                continue
            tmp_f = np.divide(d['values'][:,itype,ia],np.sum(d['values'][:,itype,:],axis=-1),
                             where = np.sum(d['values'][:,itype,:],axis=-1) >0)
#             tmp_f = d['values'][:,itype,ia] / np.sum(d['values'][:,itype,:],axis=-1)
            ax.plot(d['times'][tmin:tmax],tmp_f[tmin:tmax],label=typ)
        
        
        ax.plot(dsub['times'][tmin:tmax],stsub[:,ia][tmin:tmax],'--',lw=0.5,label='substrates')
        ax.plot(dsub['times'][tmin:tmax],stsub_corr[:,ia][tmin:tmax],'--',lw=0.5,label='substrates C/2')
        xmin,xmax = ax.get_xlim()
        ax.hlines([smin[ia],smin_comp[ia]],xmin,xmax,colors=['k','k'],linestyles='--')
        ax.grid()
        ax.legend(fontsize=9)
        ax.set_ylim(0,1)

In [ ]:
%matplotlib inline
scal = 1.0 / eco.microbes.grid_size
fig = plt.figure(figsize=(16,8))
dat = np.sum(tla_mic['microbes']['values'][:,:,:],axis=-1)
sdat = np.sum(dat,axis=1)
dat = dat / sdat[:,np.newaxis]
plt.plot(tla_mic['microbes']['times'],dat,label=eco.microbes.names)
plt.legend()
plt.xlabel('Time')
plt.ylabel('Total mass relative abundancy')
plt.grid()

In [ ]:
fig, axes = plt.subplots(3,1, figsize=(16,18))
scal = 1.0 / eco.microbes.grid_size
for ia,ax in enumerate(axes):
    for k,d in tla_mic.items():
        for itax, micname in enumerate(eco.microbes.names):
            if (itax%2 ==0):
                ls = '--'
            else:
                ls = '-'
            ax.plot(d['times'],scal*d['values'][:,itax,ia],ls,label=micname)
    ax.legend(ncol=2, fontsize=8)
    ax.grid()
    ax.set_title('Taxa Biomass {}'.format(['C','N','P'][ia]))